# This example notebook, uses the provided OGIP files, to plot the Light Curves


One can follow the tutorial from gammapy here https://docs.gammapy.org/0.18.2/tutorials/light_curve.html

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from pathlib import Path

import astropy.units as u
from astropy.time import Time
from astropy.io import fits
from astropy.table import Table

import numpy as np

from gammapy.modeling import Fit
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    LogParabolaSpectralModel,
    create_crab_spectral_model,
    SkyModel,
)
from gammapy.datasets import Datasets, SpectrumDataset, SpectrumDatasetOnOff, FluxPointsDataset

from gammapy.estimators import FluxPointsEstimator, FluxPoints, LightCurveEstimator

# 1. Get the OGIP files and make some selections if need be

In [ ]:
base_dir="/fefs/aswg/workspace/analysis-school-2022/"
dir_path="DL3/Crab_src_indep/" # DL3/BLLac_src_dep

ogip_path=Path(base_dir+dir_path+"OGIP/")
plot_path=Path(base_dir+dir_path+"plots/")

ogip_path.mkdir(exist_ok=True)
plot_path.mkdir(exist_ok=True)

In [ ]:
# Using the full standard_dataset as provided
use_standard_data = True

In [ ]:
# read all obs ids of the pha files in the given directory
if use_standard_data:
    obs_ids = [2967, 2968, 2969, 2970, 2971, 2972, 2973, 2974, 2975, 2976, 2977] # For Crab
    # [5552, 5553, 5554, 5555, 5556, 5557, 5558, 5559] # for BL Lac
else:
    # Get all the OGIP files in the provided location
    obs_ids = []
    pha_files = list(ogip_path.glob("pha_obs*.fits"))
    
    for p in pha_files:
        run = int(p.name[7:-5])
        obs_ids.append(run)
    obs_ids = np.sort(np.array(obs_ids))

In [ ]:
# Generate the Datasets object back from the OGIP files
datasets = Datasets()
for obs in obs_ids:
    file = ogip_path / f"pha_obs{obs}.fits"
    datasets.append(SpectrumDatasetOnOff.from_ogip_files(file))

In [ ]:
fits.open(ogip_path / f"pha_obs{obs_ids[1]}.fits").info()

In [ ]:
Table.read(ogip_path / f"pha_obs{obs_ids[1]}.fits", hdu="SPECTRUM").meta

In [ ]:
Table.read(ogip_path / f"pha_obs{obs_ids[1]}.fits", hdu="REGION").meta

# 2. Get some parameters to use from the datasets

In [ ]:
t_start = []
t_stop = []
tot_time = []
obj_name = []

for obs in obs_ids:
    file = ogip_path / f"pha_obs{obs}.fits"
    t = Table.read(file, hdu="GTI")
    r = Table.read(file, hdu="REGION").meta
    
    t_start.append(t["START"][0])
    t_stop.append(t["STOP"][0])
    tot_time.append(t["STOP"][0] - t["START"][0])
    
    obj_name.append(r["OBJECT"])
print(obs_ids)

t_start = np.sort(np.array(t_start))
t_stop = np.sort(np.array(t_stop))
tot_time = np.array(tot_time)
print(tot_time.sum()/3600)

obj_name = np.unique(np.array(obj_name))[0]

In [ ]:
e_reco_edges = datasets[0].counts.geom.axes["energy"].edges

In [ ]:
# Energy range for estimating the Light Curve.
# One can play with different energy bins to check the different LCs
e_fit_min = 0.01 * u.TeV
e_fit_max = 40 * u.TeV

# 3. Get Pivot energy to fix the reference energy and define the Spectrum Model

In [ ]:
# Find pivot (decorrelation) energy for a Power Law model to get the reference energy for Log Parabola model
def get_pivot_energy(datasets, e_ref, e_edges, obj_name):
    """
    Using Power Law spectral model with the given reference energy and
    get the decorrelation energy of the fit, within the fit energy range, e_edges
    """
    spectral_model = PowerLawSpectralModel(
        index=2, amplitude=2e-11 * u.Unit("cm-2 s-1 TeV-1"), reference=e_ref
    )
    model = SkyModel(spectral_model=spectral_model, name=obj_name)
    dataset_check = datasets
    model_check = model.copy()

    # Stacked dataset method
    model_final_stacked = model_check.copy()
    stacked_dataset = Datasets(dataset_check).stack_reduce()
    stacked_dataset.models = model_final_stacked

    fit_stacked = Fit(stacked_dataset)
    result_stacked = fit_stacked.run()
    model_best_stacked = model_final_stacked.copy()

    fpe_stacked = FluxPointsEstimator(energy_edges=e_edges)
    flux_points_stacked = fpe_stacked.run(datasets=stacked_dataset)
    
    flux_points_dataset = FluxPointsDataset(
        data=flux_points_stacked, models=model_best_stacked
    )
    return model_best_stacked.spectral_model.pivot_energy


In [ ]:
%%time
ref = get_pivot_energy(datasets, 0.4 * u.TeV, e_reco_edges, obj_name)
print(ref.to_value(u.GeV))

In [ ]:
# Final spectral model of Log Parabola, to be used for estimating the LC.
# One can try different Spectral Models as well.
# Be careful in the choice of Spectral Model being used for the 2 examples presented here

# Crab
spectral_model_lp = LogParabolaSpectralModel(
        amplitude = 5e-12 * u.Unit('cm-2 s-1 TeV-1'),
        reference = ref,
        alpha = 2 * u.Unit(''),
        beta = 0.1 * u.Unit('')
)
model_lp = SkyModel(spectral_model=spectral_model_lp, name=obj_name)

# BL Lac
spectral_model_lp_bllac = LogParabolaSpectralModel(
        amplitude = 3e-8 * u.Unit('cm-2 s-1 TeV-1'),
        reference = 0.1 * u.TeV,
        alpha = 2 * u.Unit(''),
        beta = 0.2 * u.Unit('')
)
model_lp_bllac = SkyModel(spectral_model=spectral_model_lp_bllac, name=obj_name)

In [ ]:
params=model_lp.to_dict()['spectral']['parameters']
# params=model_lp_bllac.to_dict()['spectral']['parameters']
params

# 4. Estimate the LC

In [ ]:
%%time
lc_maker_1d = LightCurveEstimator(
    energy_edges=[e_fit_min, e_fit_max], reoptimize=True, source=obj_name
)

In [ ]:
# Need the time start and end
t_start = Time(t_start, format='unix')
t_stop = Time(t_stop, format='unix')

t_day = np.unique(np.rint(t_start.mjd))

# To make the range night-wise, keep the MJD range in half integral values
t_range = [Time([t-0.5, t+0.5], format="mjd", scale="utc") for t in t_day]

lc_maker_night_wise = LightCurveEstimator(
    energy_edges=[e_fit_min, e_fit_max], 
    time_intervals=t_range,
    reoptimize=True, source=obj_name
)

In [ ]:
dataset_lc = datasets
model_lc = model_lp

for data in dataset_lc:
    data.models = model_lc

lc_1d = lc_maker_1d.run(dataset_lc)
lc_night = lc_maker_night_wise.run(dataset_lc)

In [ ]:
# Check the various column data of the Light Curve object
lc_1d.table

In [ ]:
# If there are more than 1 night of data, one can see the integrated light curve for each night
lc_night.table

# 5. Plot the Light Curve

In [ ]:
# Calculate & plot Crab reference flux
crab = create_crab_spectral_model("magic_lp")
crab.amplitude.error, crab.alpha.error, crab.beta.error = 0.03e-11 * u.Unit("cm-2 s-1 TeV-1"), 0.01, 0.01/np.log(10) # https://doi.org/10.1016/j.jheap.2015.01.002

flux_crab, flux_crab_error = crab.integral_error(0.095 * u.TeV, 100 * u.TeV) # E_min to E_max
print(flux_crab, flux_crab_error)

In [ ]:
fig_lc = plt.figure(figsize=(8,10))

gs2 = GridSpec(10, 4)

gs2.update(hspace=0.1)
args1 = [gs2[:5,:]]
args2 = [gs2[5:,:]]

fig_gs1 = fig_lc.add_subplot(*args1)
fig_gs2 = fig_lc.add_subplot(*args2, sharey=fig_gs1)

lc_1d.plot(
    ax=fig_gs1,
    time_format='mjd', marker="o", label="LST-1"
)
fig_gs1.axhline(
    flux_crab.to_value("cm-2 s-1"), c='red', ls='--', 
    label='Crab (MAGIC, JHEAp 2015)'
)
fig_gs1.axhspan(
    (flux_crab - flux_crab_error).to_value("cm-2 s-1"), 
    (flux_crab + flux_crab_error).to_value("cm-2 s-1"), 
    alpha=0.2, color='tab:orange'
)

fig_gs1.grid(which='both')
fig_gs1.set_title(
    f'LC LST-1 {obj_name}: Run-wise {tot_time.sum()/3600:.2f} hrs, night-wise {len(t_day)} nights'
)
fig_gs1.legend()
fig_gs1.get_yaxis().get_offset_text().set_position((-0.06,1))

lc_night.plot(
    ax=fig_gs2,
    time_format='iso', marker="o", label="LST-1"
)
fig_gs2.grid(which='both')
fig_gs2.get_yaxis().get_offset_text().set_position((-0.06,1))
fig_gs2.legend()